## Jos neki resursi

- [ ] [gsitechnology.com - A Beginner’s Guide To Segmentation In Satellite Images: Walking Through Machine Learning Techniques For Image Segmentation And Applying Them To Satellite Imagery](https://www.gsitechnology.com/Beginners-Guide-to-Segmentation-in-Satellite-Images)


- [fegu856.com - Getting started with PyTorch: 1- Linear regression](http://www.fregu856.com/post/19apr/)

- [freecodecamp - What Is a Convolutional Neural Network? A Beginner's Tutorial for Machine Learning and Deep Learning](https://www.freecodecamp.org/news/convolutional-neural-network-tutorial-for-beginners/)

<br>

---
#### za ispitat

- [ ] [kaggle.com - Segmentation in PyTorch using convenient tools](https://www.kaggle.com/artgor/segmentation-in-pytorch-using-convenient-tools)   
    - understanding clouds from satelite images
    - nisam siguran za ovaj 
- [ ] [towardsdatascience.com - Semantic hand segmentation using Pytorch](https://towardsdatascience.com/semantic-hand-segmentation-using-pytorch-3e7a0a0386fa)
    - harder than **object detection**
    - simpler than instance segmentation
    
    
- github 2 projekta
     - [torchsat](https://github.com/sshuair/torchsat)
         - python, docker
     - [satellite-building-detection.pytorch](https://github.com/YuansongFeng/satellite-building-detection.pytorch)
         - archived
         - jupyter notebook

# semantic segmentation - pytorch


- [PyTorch for Beginners: Semantic Segmentation using torchvision](https://learnopencv.com/pytorch-for-beginners-semantic-segmentation-using-torchvision/)
    - [pytorch docs - FCN-RESNET101](https://pytorch.org/hub/pytorch_vision_fcn_resnet101/)
    - [RESNET](https://pytorch.org/hub/pytorch_vision_resnet/)

When we group pixels based on color, texture, or other low level primitives, we call these perceptual groups **superpixels**

In [ ]:
from torchvision import models

# a pretrained model of FCN
# pretrained=True - download model if not already present
fcn = models.segmentation.fcn_resnet101(pretrained=True).eval()

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import torch

img = Image.open('./img/bird.jpg')
plt.imshow(img)
plt.show()

### 3.2.3. Pre-process the image

uh

In [ ]:
# Apply the transformations needed
import torchvision.transforms as T
trf = T.Compose([T.Resize(256),
                 T.CenterCrop(300),  # na 300 je ok
                 T.ToTensor(), 
                 T.Normalize(mean = [0.485, 0.456, 0.406], 
                             std = [0.229, 0.224, 0.225])])
inp = trf(img).unsqueeze(0)

In [ ]:
# Pass the input through the net
out = fcn(inp)['out']
print (out.shape)

In [ ]:
torch.Size([1, 21, 224, 224])

In [ ]:
import numpy as np
om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
print(om.shape)

In [ ]:
print (np.unique(om))

above is a 2D image where each pixel corresponds to a class

and now, it's tiime. the last thing: take this 2D img and convert it into a segmentation map where each class label is converted into an RGB color

### Decode Output

the following function does that

In [ ]:
# Define the helper function
def decode_segmap(image, nc=21):
  
  label_colors = np.array([(0, 0, 0),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (128, 0, 0), (0, 128, 0), (128, 128, 0), (0, 204, 204), (128, 0, 128),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (0, 128, 128), (128, 128, 128), (64, 0, 0), (192, 0, 0), (64, 128, 0),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (192, 128, 0), (64, 0, 128), (192, 0, 128), (64, 128, 128), (192, 128, 128),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0), (0, 64, 128)])

  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
  
  for l in range(0, nc):
    idx = image == l
    r[idx] = label_colors[l, 0]
    g[idx] = label_colors[l, 1]
    b[idx] = label_colors[l, 2]
    
  rgb = np.stack([r, g, b], axis=2)
  return rgb

In [ ]:
rgb = decode_segmap(om)
plt.imshow(rgb)
plt.show()

In [ ]:
def segment(net, path):
  img = Image.open(path)
  plt.imshow(img); plt.axis('off'); plt.show()
  # Comment the Resize and CenterCrop for better inference results
  trf = T.Compose([T.Resize(256), 
                   T.CenterCrop(400), 
                   T.ToTensor(), 
                   T.Normalize(mean = [0.485, 0.456, 0.406], 
                               std = [0.229, 0.224, 0.225])])
  inp = trf(img).unsqueeze(0)
  out = net(inp)['out']
  om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
  rgb = decode_segmap(om)
  plt.imshow(rgb); plt.axis('off'); plt.show()

In [ ]:
segment(fcn, './img/white-tailed_Tropicbird.jpg')

it's ok


In [ ]:
segment(fcn, './img/bicycle.jpg')

miracles

### DeepLab is supposedly better

<br>


<!-- - ![pre_trained_model_accuracy](pre_trained_model_accuracy.png) -->
- [TORCHVISION.MODELS](https://pytorch.org/docs/stable/torchvision/models.html?highlight=faster%20rcnn)
![segmentation models: FCN101 vs DeepLabV3 ResNet101](./img/segmentation_models.png)


- [pyimagesearch.com - IoU](https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/) - intersection over union
    - measure the accuracy of an object detector on a particular dataset
    - good grade is at 0.73, allegedly
    - poor 0.4
    - excellent 0.93
    - ![calculate IoU](./img/IoU.png)
    
<br>

- [blog fergu856.com - PyTorch Implementation of DeepLabV3](http://www.fregu856.com/project/deeplabv3/)
    - autonomus driving

In [ ]:
dlab = models.segmentation.deeplabv3_resnet101(pretrained=1).eval()

In [ ]:
segment(dlab, './img/bicycle.jpg')

uh


maybe the images were too difficult

In [ ]:
segment(dlab, './img/boat.jfif')